In [1]:
import pandas as pd 
import csv as csv 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [2]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
data["Sex"] = data["Sex"].map({"female" : 0, "male" : 1})
test["Sex"] = test["Sex"].map({"female" : 0, "male" : 1})

In [4]:
data["Embarked"] = data["Embarked"].map({"Q" : 2, "S" : 3, "C" : 4})
test["Embarked"] = test["Embarked"].map({"Q" : 2, "S" : 3, "C" : 4})

In [5]:
data["Age"].fillna(data["Age"].mean(), inplace = True)
test["Age"].fillna(test["Age"].mean(), inplace = True)

In [6]:
data["SibSp"].fillna(data["SibSp"].mean(), inplace = True)
test["SibSp"].fillna(test["SibSp"].mean(), inplace = True)

In [7]:
data["Parch"].fillna(data["Parch"].mean(), inplace = True)
test["Parch"].fillna(test["Parch"].mean(), inplace = True)

In [8]:
data["Fare"].fillna(data["Fare"].mean(), inplace = True)
test["Fare"].fillna(test["Fare"].mean(), inplace = True)

In [9]:
data["Embarked"].fillna(data["Embarked"].mean(), inplace = True)
test["Embarked"].fillna(test["Embarked"].mean(), inplace = True)

In [10]:
data = data.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis = 1)
test = test.drop(['Name', 'Ticket', 'Cabin'], axis = 1)

In [11]:
target = "Survived"
x_train = data.drop("Survived", axis = 1)
y_train = data["Survived"]
x_test2 = test.drop("PassengerId", axis = 1).copy()

In [12]:
x_train, x_test, y_train, y_test = train_test_split(
x_train, y_train, test_size = 0.3, random_state = 0)

In [13]:
classifier = RandomForestClassifier(n_estimators = 100)
classifier.fit(x_train, y_train)

y_predict = classifier.predict(x_test2)
#confusion_matrix(y_test,y_predict)

In [14]:
final = pd.read_csv('test.csv', header = 0)
final.shape
final = final[['PassengerId']]
final['Survived'] = y_predict
final.to_csv("SubmitFinal.csv", index = False)
final

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0


In [15]:
import random
import operator

import numpy as np
import matplotlib.pyplot as plt

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [17]:
pset = gp.PrimitiveSetTyped("main", [float, float, float, float, 
                                     bool, bool, bool, bool, bool, bool, bool, bool], bool)
pset.addPrimitive(operator.xor, [bool, bool], bool)
pset.addPrimitive(operator.and_, [bool, bool], bool)
pset.addPrimitive(operator.or_, [bool, bool], bool)
pset.addPrimitive(operator.mul, [float, float], float)
pset.addPrimitive(operator.add, [float, float], float)
pset.addPrimitive(operator.pow, [float, float], float)
pset.addPrimitive(np.subtract, [float, float], float)

In [18]:
pset.addEphemeralConstant('randFloat', lambda: random.uniform(-10, 10), ret_type=float)
pset.addEphemeralConstant('randBool', lambda: bool(random.getrandbits(1)), ret_type=bool)

In [19]:
creator.create('FitnessMin', base.Fitness, weights=(-1.,))
creator.create('Individual', gp.PrimitiveTree, fitness=creator.FitnessMin)

In [20]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=8)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

In [21]:
def evalIndividual(individual, instances, labels):
    func = toolbox.compile(expr=individual)
    error = 0
    for instance, label in zip(instances, labels):
#         print(instance, label)
        if func(instance.tolist()) != label:
            error += 1
    return (error / instances.shape[0],)

In [22]:
toolbox.register("evaluate", evalIndividual, instances=x_train.as_matrix(), labels=y_train.as_matrix())
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

In [23]:
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=12))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=12))

In [24]:
def main():
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,
                                   halloffame=hof, verbose=True)
    # print log
    return pop, log, hof
main()

   	      	                        fitness                         	              size             
   	      	--------------------------------------------------------	-------------------------------
gen	nevals	avg     	max     	min     	std     	avg    	max	min	std    
0  	300   	0.506265	0.617978	0.380417	0.107597	19.9133	209	2  	29.1308
1  	162   	0.438283	0.616372	0.380417	0.0909583	21.36  	202	1  	25.0898
2  	166   	0.410107	0.611557	0.359551	0.0660342	22.04  	205	1  	25.8009
3  	162   	0.403676	0.626003	0.359551	0.0574266	23.91  	204	1  	26.8948
4  	155   	0.404248	0.648475	0.35313 	0.059805 	24.8767	152	1  	20.9008
5  	156   	0.396816	0.611557	0.35313 	0.0520643	30.4667	152	1  	21.1657
6  	171   	0.391798	0.617978	0.346709	0.0533044	39.2567	119	1  	19.5661
7  	179   	0.388571	0.626003	0.346709	0.0614101	47.6267	90 	1  	15.3156
8  	166   	0.375966	0.617978	0.345104	0.0503577	53.7933	85 	1  	10.6679
9  	183   	0.372338	0.617978	0.345104	0.05022  	56.0067	88 	20 	10.6107
10 	156   

([[<deap.gp.Primitive at 0x10d136db8>,
 [{'gen': 0, 'nevals': 300},
  {'gen': 1, 'nevals': 162},
  {'gen': 2, 'nevals': 166},
  {'gen': 3, 'nevals': 162},
  {'gen': 4, 'nevals': 155},
  {'gen': 5, 'nevals': 156},
  {'gen': 6, 'nevals': 171},
  {'gen': 7, 'nevals': 179},
  {'gen': 8, 'nevals': 166},
  {'gen': 9, 'nevals': 183},
  {'gen': 10, 'nevals': 156},
  {'gen': 11, 'nevals': 183},
  {'gen': 12, 'nevals': 165},
  {'gen': 13, 'nevals': 180},
  {'gen': 14, 'nevals': 152},
  {'gen': 15, 'nevals': 174},
  {'gen': 16, 'nevals': 173},
  {'gen': 17, 'nevals': 156},
  {'gen': 18, 'nevals': 172},
  {'gen': 19, 'nevals': 181},
  {'gen': 20, 'nevals': 177},
  {'gen': 21, 'nevals': 161},
  {'gen': 22, 'nevals': 170},
  {'gen': 23, 'nevals': 155},
  {'gen': 24, 'nevals': 153},
  {'gen': 25, 'nevals': 203},
  {'gen': 26, 'nevals': 153},
  {'gen': 27, 'nevals': 182},
  {'gen': 28, 'nevals': 153},
  {'gen': 29, 'nevals': 161},
  {'gen': 30, 'nevals': 167},
  {'gen': 31, 'nevals': 174},
  {'gen': 3